In [1]:
%pip install ../dependencies/boto3-1.26.140-py3-none-any.whl --quiet
%pip install ../dependencies/botocore-1.29.140-py3-none-any.whl --quiet
%pip install python-dotenv boto3 langchain==0.0.249 PyPdf  --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
import boto3
boto_bedrock = boto3.client(
    service_name='bedrock',
    endpoint_url='https://bedrock.us-west-2.amazonaws.com'
)

from langchain.embeddings import BedrockEmbeddings

br_embeddings = BedrockEmbeddings(client=boto_bedrock)
# br_embeddings = BedrockEmbeddings(model_id="amazon.titan-tg1-large", 
#               model_kwargs ={
#                 "textGenerationConfig": {
#                    "maxTokenCount": 4096,
#                    "stopSequences": [],
#                    "temperature":0,
#                    "topP":1
#                 },
#               },
#               client=boto_bedrock)

In [27]:
!wget https://docs.spring.io/spring-framework/docs/6.0.8/reference/pdf/spring-framework.pdf
!wget https://docs.spring.io/spring-boot/docs/3.1.2/reference/pdf/spring-boot-reference.pdf

--2023-08-17 16:51:37--  https://docs.spring.io/spring-framework/docs/6.0.8/reference/pdf/spring-framework.pdf
Resolving docs.spring.io (docs.spring.io)... 104.20.3.65, 104.20.2.65
Connecting to docs.spring.io (docs.spring.io)|104.20.3.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20829650 (20M) [application/pdf]
Saving to: ‘spring-framework.pdf’

spring-framework.pd 100%[===================>]  19.86M  10.1MB/s    in 2.0s    

2023-08-17 16:51:39 (10.1 MB/s) - ‘spring-framework.pdf’ saved [20829650/20829650]

--2023-08-17 16:51:39--  https://docs.spring.io/spring-boot/docs/3.1.2/reference/pdf/spring-boot-reference.pdf
Resolving docs.spring.io (docs.spring.io)... 104.20.3.65, 104.20.2.65
Connecting to docs.spring.io (docs.spring.io)|104.20.3.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17299408 (16M) [application/pdf]
Saving to: ‘spring-boot-reference.pdf’

spring-boot-referen 100%[===================>]  16.50M  9.92MB/s    

In [4]:
# from langchain.document_loaders import PyPDFLoader

# # Multiple file -  Load PDF
# loaders = [
#     PyPDFLoader("https://docs.spring.io/spring-framework/docs/6.0.8/reference/pdf/spring-framework.pdf"),
#     PyPDFLoader("https://docs.spring.io/spring-boot/docs/3.1.2/reference/pdf/spring-boot-reference.pdf")
#     # Add more pdf files into this array,

# ]
# docs = []
# for loader in loaders:
#     docs.extend(loader.load())

# # Split
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 2000,
#     chunk_overlap = 400,
#     separators=["\n", "\r\n", "\r", "\t", " "]
# )
# splits = text_splitter.split_documents(docs)
# # print(len(splits))


In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_pdf_files(file_names, chunk_size=512, chunk_overlap=50, separators=["\n", "\r\n", "\r", "\t", " "]):
    """
    Load and split multiple PDF files into smaller chunks of text.

    Args:
    file_names (list): A list of file names to load.
    chunk_size (int): The maximum size of each text chunk.
    chunk_overlap (int): The number of characters to overlap between adjacent text chunks.
    separators (list): A list of characters to use as separators when splitting the text.

    Returns:
    A list of text chunks.
    """
    # Load PDF files
    loaders = [PyPDFLoader(file_name) for file_name in file_names]
    docs = []
    for loader in loaders:
        docs.extend(loader.load())

    # Split text into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=separators
    )
    splits = text_splitter.split_documents(docs)

    return splits

In [4]:
file_names = ["spring-framework.pdf", "spring-boot-reference.pdf"]
splits = load_and_split_pdf_files(file_names)

print(len(splits))

9981


In [6]:
len(splits)

9981

In [7]:

# import pandas as pd
# myframe = pd.DataFrame()
# myframe['page_content'] = list(map(lambda x:x.page_content, splits))
# myframe['metadata'] = list(map(lambda x:x.metadata, splits))
# myframe.to_parquet('documents/spring_docs.parquet')

# docs_from_parquet = pd.read_parquet('documents/spring_docs.parquet')
# from langchain.schema.document import Document
# # fill dataframe with one row per object, one attribute per column
# docs_from_parquet['docs'] = docs_from_parquet.apply(lambda row: Document(page_content=row.page_content, metadata=row.metadata), axis = 1)

In [7]:
import pandas as pd
from langchain.schema.document import Document

def save_documents_to_parquet(documents, file_path):
    """
    Save a list of documents to a Parquet file.

    Args:
    documents (list): A list of Document objects.
    file_path (str): The file path to save the Parquet file to.
    """
    # Create a DataFrame from the list of documents
    df = pd.DataFrame({
        'page_content': [doc.page_content for doc in documents],
        'metadata': [doc.metadata for doc in documents]
    })

    # Save the DataFrame to a Parquet file
    df.to_parquet(file_path)

def read_documents_from_parquet(file_path):
    """
    Read a list of documents from a Parquet file.

    Args:
    file_path (str): The file path to read the Parquet file from.

    Returns:
    A list of Document objects.
    """
    # Read the DataFrame from the Parquet file
    df = pd.read_parquet(file_path)

    # Create a list of Document objects from the DataFrame
    documents = [Document(page_content=row['page_content'], metadata=row['metadata']) for _, row in df.iterrows()]

    return documents

In [15]:
# Save the list of documents to a Parquet file
save_documents_to_parquet(splits, 'documents/spring_docs.parquet')

# Read the list of documents from the Parquet file
documents = read_documents_from_parquet('documents/spring_docs.parquet')

# Print the first document
print(documents[0])

page_content='Spring Framework Documentation\nRod Johnson, Juergen Hoeller, Keith Donald, Colin Sampaleanu, Rob Harrop,\nThomas Risberg, Alef Arendsen, Darren Davison, Dmitriy Kopylenko, Mark\nPollack, Thierry Templier, Erwin Vervaet, Portia Tung, Ben Hale, Adrian Colyer,\nJohn Lewis, Costin Leau, Mark Fisher, Sam Brannen, Ramnivas Laddad, Arjen\nPoutsma, Chris Beams, Tareq Abedrabbo, Andy Clement, Dave Syer, Oliver\nGierke, Rossen Stoyanchev, Phillip Webb, Rob Winch, Brian Clozel, Stephane' metadata={'page': 0, 'source': 'spring-framework.pdf'}


In [17]:
documents[:5]

[Document(page_content='Spring Framework Documentation\nRod Johnson, Juergen Hoeller, Keith Donald, Colin Sampaleanu, Rob Harrop,\nThomas Risberg, Alef Arendsen, Darren Davison, Dmitriy Kopylenko, Mark\nPollack, Thierry Templier, Erwin Vervaet, Portia Tung, Ben Hale, Adrian Colyer,\nJohn Lewis, Costin Leau, Mark Fisher, Sam Brannen, Ramnivas Laddad, Arjen\nPoutsma, Chris Beams, Tareq Abedrabbo, Andy Clement, Dave Syer, Oliver\nGierke, Rossen Stoyanchev, Phillip Webb, Rob Winch, Brian Clozel, Stephane', metadata={'page': 0, 'source': 'spring-framework.pdf'}),
 Document(page_content='Nicoll, Sebastien Deleuze, Jay Bryant, Mark Paluch\nVersion 6.0.8', metadata={'page': 0, 'source': 'spring-framework.pdf'}),
 Document(page_content='Table of Contents\n1. Legal . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . \xa02\n2. Spring Framework Overview . . . . . . . . . . . . . . . 

In [19]:
import pandas as pd
import numpy as np

def save_embeddings_to_parquet(embeddings, file_path):
    """
    Save a numpy array of embeddings to a Parquet file.

    Args:
    embeddings (numpy.ndarray): A numpy array of embeddings.
    file_path (str): The file path to save the Parquet file to.
    """
    # Create a DataFrame from the numpy array of embeddings
    df = pd.DataFrame(embeddings)

    # Save the DataFrame to a Parquet file
    df.to_parquet(file_path)

def load_embeddings_from_parquet(file_path):
    """
    Load a numpy array of embeddings from a Parquet file.

    Args:
    file_path (str): The file path to read the Parquet file from.

    Returns:
    A numpy array of embeddings.
    """
    # Read the DataFrame from the Parquet file
    df = pd.read_parquet(file_path)

    # Convert the DataFrame to a numpy array of embeddings
    embeddings = df.to_numpy()

    return embeddings

In [6]:
from langchain.vectorstores import FAISS

vectorstore_faiss_aws = FAISS.from_documents(
    documents=splits[:1000],
    embedding = br_embeddings, 
    #**k_args
)

print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

vectorstore_faiss_aws:created=<langchain.vectorstores.faiss.FAISS object at 0x16c3d0c10>::


In [11]:
%pip install Chromadb -qU

Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain.vectorstores import Chroma
# Load the vector DB
persist_directory = "./data/spring-reference"
# import tiktoken
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=br_embeddings,
    persist_directory=persist_directory
)


print("Completed Loading the PDF into the Vector DB with a resulting Vector Collection Count - " , vectordb._collection.count())
# 500 documents in the collection : 1m: 36s
# 1000 documents in the collection : 2m: 58s
# 2000 documents in the collection : 5m: 18s
# 5000 documents in the collection : 14m: 36s
# 9981 documents in the collection : 29m: 52s


Completed Loading the PDF into the Vector DB with a resulting Vector Collection Count -  9981


In [ ]:
% pip install pinecone-client -qU

In [ ]:
from langchain.vectorstores import Pinecone
import pinecone

# initialize pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

index_name = "langchain-demo"

# First, check if our index already exists. If it doesn't, we create it
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
      name=index_name,
      metric='cosine',
      dimension=1536  
)
# The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [17]:
import datetime
def get_qa_chain():
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=br_embeddings
    )

    print(vectordb._collection.count())
    llm = Bedrock(model_id="anthropic.claude-v2", client=boto_bedrock, model_kwargs={"max_tokens_to_sample":2000})
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever()
    )

    return qa_chain


In [12]:
import datetime
from langchain.vectorstores import Chroma
from langchain.llms import Bedrock
from langchain.chains import RetrievalQA
def get_qa_chain_for_titan():
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=br_embeddings
    )

    print(vectordb._collection.count())
    llm = Bedrock(model_id="amazon.titan-tg1-large", client=boto_bedrock, model_kwargs={"maxTokenCount":2000})
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever()
    )

    return qa_chain


# CREATE QA Chain
qa_chain_titan = get_qa_chain_for_titan()


9981


In [20]:

question1 = "What is latest Spring Framework version and spring boot version you know about?"
result = qa_chain_titan({"query": question1})
print(result["result"])

question1 = "tell me more about a  A step-by-step migration guide for Boot 1.0 and Java to Boot 2.0 and Kotlin?"
result = qa_chain_titan({"query": question1})
print(result["result"])

 The latest stable version of the Spring Framework is 6.2.2 and the latest stable version of Spring Boot is 2.6.3.
 a step-by-step migration guide for Boot 1.0 and Java to Boot 2.0 and Kotlin, is a sample project that demonstrates how to migrate an existing Spring Boot 1.0 application to Boot 2.0 and Kotlin. The project includes detailed instructions and code examples to help you upgrade your application step by step.


In [21]:
qa_chain = get_qa_chain()

question1 = "What is latest Spring Framework version and spring boot version you know about?"
result = qa_chain({"query": question1})
print(result["result"])

question1 = "tell me more about a  A step-by-step migration guide for Boot 1.0 and Java to Boot 2.0 and Kotlin?"
result = qa_chain({"query": question1})
print(result["result"])


9981
 Based on the SDK output, the latest available version of Spring Boot is 3.1.2. The latest version of Spring Framework is 6.0, which is used in Spring Boot 3.x releases. Spring Framework 6.0 upgrades to Jakarta EE 9 APIs and is fully compatible with Java EE servers like Tomcat 10+, Jetty 11, and Undertow 2.3.
 Here are some key details about the spring-kotlin-deepdive sample project, which provides a step-by-step migration from Spring Boot 1.0 + Java to Spring Boot 2.0 + Kotlin:

- It demonstrates migrating a simple Spring Boot 1.0 Java application to Spring Boot 2.0 Kotlin in small incremental steps. 

- Each step is tagged in Git so you can see the exact changes between steps.

- It starts with a basic Spring Boot 1.0 Java app and eventually ends up as a Spring Boot 2.0 Kotlin app, while retaining the same functionality.

- Along the way it converts Java configuration to Kotlin DSL, Java classes to Kotlin classes, converts tests from JUnit 4 + Mockito to JUnit 5, Switches from M